In [ ]:
%load_ext autoreload

%autoreload 2
%matplotlib inline

import datetime as dt

import matplotlib
import matplotlib.cm as mplcm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytz
from history import History

pd.set_option("display.max_columns", 100)

## Download and store history

In [ ]:
hist_daily = History(interval="day")
hist_daily.update(reload=True)
hist_daily.data

In [ ]:
import pytz

a = hist_daily.data
a[
    a["time"] >= dt.datetime.now(pytz.timezone("Europe/Moscow")) - dt.timedelta(weeks=1)
].figi.unique()

In [ ]:
a = hist_daily.data
a = a[
    a["time"] >= dt.datetime.now(pytz.timezone("Europe/Moscow")) - dt.timedelta(weeks=1)
]
a["date"] = a["time"].dt.date
a.groupby(["ticker", "date"]).count()

In [ ]:
ticker = "FXGD"
end = dt.datetime.now(dt.timezone.utc)
start = end - dt.timedelta(weeks=52)
interval = "day"

ticker_history = hist_daily.get_ticker_history(ticker, start, end, interval)
ticker_history

In [ ]:
plt.figure()
plt.plot(ticker_history["time"].values, ticker_history["c"])
plt.title(ticker)

In [ ]:
etfs_history, etf_tickers = hist_daily.get_etfs_history()
print(etf_tickers)

In [ ]:
data = hist_daily.data
tickers = hist_daily.tickers
# print(data['time'].dtype)

M = len(hist_daily.tickers)
cm = plt.get_cmap("gist_rainbow")
cNorm = colors.Normalize(vmin=0, vmax=M - 1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)


def get_last_not_nan(lst):
    for el in lst[::-1]:
        if not np.isnan(el):
            return el

    return np.nan


plt.figure(figsize=(20, 10))
ax = plt.gca()
ax.set_prop_cycle(color=[scalarMap.to_rgba(i) for i in range(M)])
for ticker in tickers:
    filter = data.ticker == ticker
    t = data.loc[filter, "time"].values
    y = data.loc[filter, "c"].values  # / get_last_not_nan(data[ticker+'_c']) * 100
    #     print(ticker, y)
    plt.plot(t, y, label=ticker)

plt.ylabel("Price")
plt.xlabel("Time")
plt.legend()

plt.figure(figsize=(20, 10))
ax = plt.gca()
ax.set_prop_cycle(color=[scalarMap.to_rgba(i) for i in range(M)])
for ticker in tickers:
    filter = data.ticker == ticker
    t = data.loc[filter, "time"].values
    y = data.loc[filter, "c"] / get_last_not_nan(data.loc[filter, "c"]) * 100
    plt.plot(t, y, label=ticker)

plt.ylabel("Price, % (100%=now)")
plt.xlabel("Time")
plt.legend()

## Basic calculate_statistics

In [ ]:
hist_daily.calculate_statistics()

## Basic recommend

In [ ]:
hist_daily.recommend_simple()

In [ ]:
hist_daily.recommend_other()